In [52]:
import re
import requests
from functools import cmp_to_key as keyFromComparison

In [53]:
VERSION_COMP_RE = re.compile(
    r"""
    ([0-9]*)
    ([a-z]*)
    (-?)
    """,
    re.X,
)


def splitComp(c):
    return [m for m in VERSION_COMP_RE.findall(c) if m[0] or m[1]]


def makeComps(v):
    return [splitComp(c) for c in v.split(".")]


def versionCompare(v1, v2):
    v1comps = makeComps(v1)
    v2comps = makeComps(v2)

    nV2 = len(v2comps)

    for i, c1 in enumerate(v1comps):
        if i >= nV2:
            return 1

        c2 = v2comps[i]
        nC2 = len(c2)
        for j, s1 in enumerate(c1):
            if j >= nC2:
                return 1

            s2 = c2[j]
            if s1 < s2:
                return -1
            if s1 > s2:
                return 1
    return 0


def getVersionKeyFunc():
    return keyFromComparison(versionCompare)

versionKey = getVersionKeyFunc()

In [41]:
BACKEND = "https://api.github.com/repos/{org}/{repo}/tags"
ORG = "Smithsonian"
REPO = "dpo-voyager"
HEADERS = dict(Accept="application/vnd.github+json")

In [54]:
def getTags(org, repo):
    url = BACKEND.format(org=ORG, repo=REPO)

    results = []
    good = True
    
    while True:
        response = requests.get(url, headers=HEADERS)

        if not response.ok:
            good = False
            break

        results.extend(response.json())

        links = response.links
        nxt = links.get("next", None)

        if not nxt:
            break

        nxtUrl = nxt.get("url", None)

        if not nxtUrl:
            break

        url = nxtUrl
        print(f"Continuing with {url}")

    return (good, results)

In [58]:
good, rawResults = getTags(ORG, REPO)

Continuing with https://api.github.com/repositories/151541728/tags?page=2
Continuing with https://api.github.com/repositories/151541728/tags?page=3
Continuing with https://api.github.com/repositories/151541728/tags?page=4


In [59]:
results = list(reversed(sorted([(r["name"].lstrip("v"), r["zipball_url"]) for r in rawResults], key=lambda x: versionKey(x[0]))))

In [60]:
len(results)

102

In [61]:
results

[('0.9.0',
  'https://api.github.com/repos/Smithsonian/dpo-voyager/zipball/refs/tags/v0.9.0'),
 ('0.8.1.2',
  'https://api.github.com/repos/Smithsonian/dpo-voyager/zipball/refs/tags/v0.8.1.2'),
 ('0.8.1',
  'https://api.github.com/repos/Smithsonian/dpo-voyager/zipball/refs/tags/v0.8.1'),
 ('0.8.1.1',
  'https://api.github.com/repos/Smithsonian/dpo-voyager/zipball/refs/tags/v0.8.1.1'),
 ('0.8.0',
  'https://api.github.com/repos/Smithsonian/dpo-voyager/zipball/refs/tags/v0.8.0'),
 ('0.7.5',
  'https://api.github.com/repos/Smithsonian/dpo-voyager/zipball/refs/tags/v0.7.5'),
 ('0.7.1',
  'https://api.github.com/repos/Smithsonian/dpo-voyager/zipball/refs/tags/v0.7.1'),
 ('0.7.0',
  'https://api.github.com/repos/Smithsonian/dpo-voyager/zipball/refs/tags/v0.7.0'),
 ('0.6.4',
  'https://api.github.com/repos/Smithsonian/dpo-voyager/zipball/refs/tags/v0.6.4'),
 ('0.6.3',
  'https://api.github.com/repos/Smithsonian/dpo-voyager/zipball/refs/tags/v0.6.3'),
 ('0.6.2',
  'https://api.github.com/repos

In [37]:
for r in results:
    print(f"{r['name']} at {r['zipball_url']}")

v0.56.2 at https://api.github.com/repos/Smithsonian/dpo-voyager/zipball/refs/tags/v0.56.2
v0.56.1 at https://api.github.com/repos/Smithsonian/dpo-voyager/zipball/refs/tags/v0.56.1
v0.56.0 at https://api.github.com/repos/Smithsonian/dpo-voyager/zipball/refs/tags/v0.56.0
v0.55.0 at https://api.github.com/repos/Smithsonian/dpo-voyager/zipball/refs/tags/v0.55.0
v0.54.1 at https://api.github.com/repos/Smithsonian/dpo-voyager/zipball/refs/tags/v0.54.1
v0.54.0 at https://api.github.com/repos/Smithsonian/dpo-voyager/zipball/refs/tags/v0.54.0
v0.53.0 at https://api.github.com/repos/Smithsonian/dpo-voyager/zipball/refs/tags/v0.53.0
v0.52.0 at https://api.github.com/repos/Smithsonian/dpo-voyager/zipball/refs/tags/v0.52.0
v0.51.0 at https://api.github.com/repos/Smithsonian/dpo-voyager/zipball/refs/tags/v0.51.0
v0.50.0 at https://api.github.com/repos/Smithsonian/dpo-voyager/zipball/refs/tags/v0.50.0
v0.49.0 at https://api.github.com/repos/Smithsonian/dpo-voyager/zipball/refs/tags/v0.49.0
v0.48.0 at